## 4.4 Временные ряды

In [1]:
import torch
import numpy as np

### 4.4.1 Добавляем измерение времени

In [2]:
bikes_numpy = np.loadtxt(
    "../dlwpt-code/data/p1ch4/bike-sharing-dataset/hour-fixed.csv",
    dtype=np.float32,
    delimiter=",",
    skiprows=1,
    converters={1: lambda x: float(x[8:10])}
)
bikes_numpy.shape

(17520, 17)

In [3]:
bikes = torch.from_numpy(bikes_numpy)
bikes, bikes.shape

(tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
          1.6000e+01],
         [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
          4.0000e+01],
         [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
          3.2000e+01],
         ...,
         [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
          9.0000e+01],
         [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
          6.1000e+01],
         [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
          4.9000e+01]]),
 torch.Size([17520, 17]))

### 4.4.2 Компановка данных по периоду времени

In [4]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

Изменим  форму данных, чтобы получилось три оси координат: день, час и наши 17 столбцов

In [5]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

Для получения желаемого упорядочения N x C x L необходимо транспонировать тензор.

In [6]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

### 4.4.3 Готов к обучению

In [8]:
first_day = bikes[:24]
first_day.shape

torch.Size([24, 17])

In [10]:
weather_onehot = torch.zeros(first_day.shape[0], 4)
weather_onehot.shape

torch.Size([24, 4])

In [11]:
first_day[:, 9]

tensor([1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2.,
        3., 3., 2., 2., 2., 2.])

In [13]:
weather_onehot.scatter_(
    dim=1,
    index=first_day[:, 9].unsqueeze(1).long() - 1,
    value=1.0
)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [15]:
bikes.shape, weather_onehot.shape

(torch.Size([17520, 17]), torch.Size([24, 4]))

In [14]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [16]:
torch.cat((bikes[:24], weather_onehot), 1).shape

torch.Size([24, 21])